# Introduction to SQL and Advanced Functions | Assignment (DA-AG-014)




## Question 1
**Explain the fundamental differences between DDL, DML, and DQL commands in SQL. Provide one example for each type of command.**


**Answer:**
- **DDL (Data Definition Language):** Used to create or modify database structures (tables, schemas, indexes).
  - Example:
```sql
CREATE TABLE Employees (
  EmpID INT PRIMARY KEY,
  EmpName VARCHAR(100),
  Salary DECIMAL(10,2)
);
```
- **DML (Data Manipulation Language):** Used to manipulate data (INSERT, UPDATE, DELETE).
  - Example:
```sql
INSERT INTO Employees (EmpID, EmpName, Salary) VALUES (1, 'John Doe', 45000);
```
- **DQL (Data Query Language):** Used to query/retrieve data (SELECT).
  - Example:
```sql
SELECT * FROM Employees;
```


## Question 2
**What is the purpose of SQL constraints? Name and describe three common types of constraints, providing a simple scenario where each would be useful.**


**Answer:**
Constraints ensure data integrity and correctness.
- **PRIMARY KEY:** Uniquely identifies each row and cannot be NULL. Useful for `CustomerID` in a `Customers` table.
- **FOREIGN KEY:** Enforces referential integrity between related tables. Useful when `Orders.CustomerID` references `Customers.CustomerID`.
- **UNIQUE:** Ensures all values in a column are distinct. Useful for `Email` in a `Users` table.

Example:
```sql
CREATE TABLE Customers (
  CustomerID INT PRIMARY KEY,
  Email VARCHAR(100) UNIQUE
);
```


## Question 3
**Explain the difference between LIMIT and OFFSET clauses in SQL. How would you use them together to retrieve the third page of results, assuming each page has 10 records?**


**Answer:**
- **LIMIT** restricts the number of rows returned.
- **OFFSET** skips a number of rows before starting to return rows.

To retrieve the **3rd page** (records 21–30) when each page has 10 records:
```sql
SELECT * FROM Products
LIMIT 10 OFFSET 20;
```


## Question 4
**What is a Common Table Expression (CTE) in SQL, and what are its main benefits? Provide a simple SQL example demonstrating its usage.**


**Answer:**
A CTE is a temporary named result set defined within a query. Benefits include improved readability, ability to reuse subqueries, and support for recursive queries.

Example:
```sql
WITH HighSalary AS (
  SELECT EmpName, Salary FROM Employees WHERE Salary > 50000
)
SELECT * FROM HighSalary;
```


## Question 5
**Describe the concept of SQL Normalization and its primary goals. Briefly explain the first three normal forms (1NF, 2NF, 3NF).**


**Answer:**
Normalization organizes data to reduce redundancy and improve integrity.
- **1NF:** Each column contains atomic (single) values and each row is unique.
- **2NF:** Must be in 1NF and all non-key attributes fully depend on the primary key (no partial dependencies).
- **3NF:** Must be in 2NF and no transitive dependencies (non-key attributes depending on other non-key attributes).


## Question 6
**Create a database named `ECommerceDB` and perform the following tasks:**
1. Create tables: Categories, Products, Customers, Orders
2. Insert provided records


In [ ]:
-- SQL DDL & DML statements for ECommerceDB
CREATE DATABASE IF NOT EXISTS ECommerceDB;
USE ECommerceDB;
CREATE TABLE IF NOT EXISTS Categories (
  CategoryID INT PRIMARY KEY,
  CategoryName VARCHAR(50) NOT NULL UNIQUE
);
CREATE TABLE IF NOT EXISTS Products (
  ProductID INT PRIMARY KEY,
  ProductName VARCHAR(100) NOT NULL UNIQUE,
  CategoryID INT,
  Price DECIMAL(10,2) NOT NULL,
  StockQuantity INT,
  FOREIGN KEY (CategoryID) REFERENCES Categories(CategoryID)
);
CREATE TABLE IF NOT EXISTS Customers (
  CustomerID INT PRIMARY KEY,
  CustomerName VARCHAR(100) NOT NULL,
  Email VARCHAR(100) UNIQUE,
  JoinDate DATE
);
CREATE TABLE IF NOT EXISTS Orders (
  OrderID INT PRIMARY KEY,
  CustomerID INT,
  OrderDate DATE NOT NULL,
  TotalAmount DECIMAL(10,2),
  FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);
-- Insert records
INSERT INTO Categories (CategoryID, CategoryName) VALUES
(1, 'Electronics'), (2, 'Books'), (3, 'Home Goods'), (4, 'Apparel');

INSERT INTO Products (ProductID, ProductName, CategoryID, Price, StockQuantity) VALUES
(101, 'Laptop Pro', 1, 1200.00, 50),
(102, 'SQL Handbook', 2, 45.50, 200),
(103, 'Smart Speaker', 1, 99.99, 150),
(104, 'Coffee Maker', 3, 75.00, 80),
(105, 'Novel: The Great SQL', 2, 25.00, 120),
(106, 'Wireless Earbuds', 1, 150.00, 100),
(107, 'Blender X', 3, 120.00, 60),
(108, 'T-Shirt Casual', 4, 20.00, 300);

INSERT INTO Customers (CustomerID, CustomerName, Email, JoinDate) VALUES
(1, 'Alice Wonderland', 'alice@example.com', '2023-01-10'),
(2, 'Bob the Builder', 'bob@example.com', '2022-11-25'),
(3, 'Charlie Chaplin', 'charlie@example.com', '2023-03-01'),
(4, 'Diana Prince', 'diana@example.com', '2021-04-26');

INSERT INTO Orders (OrderID, CustomerID, OrderDate, TotalAmount) VALUES
(1001, 1, '2023-04-26', 1245.50),
(1002, 2, '2023-10-12', 99.99),
(1003, 1, '2023-07-01', 145.00),
(1004, 3, '2023-01-14', 150.00),
(1005, 2, '2023-09-24', 120.00),
(1006, 1, '2023-06-19', 20.00);


## Question 7
**Generate a report showing CustomerName, Email, and the TotalNumberofOrders for each customer (include customers with 0 orders). Order by CustomerName.**


In [ ]:
-- Total orders per customer (including customers with zero orders)
SELECT c.CustomerName, c.Email, COUNT(o.OrderID) AS TotalNumberofOrders
FROM Customers c
LEFT JOIN Orders o ON c.CustomerID = o.CustomerID
GROUP BY c.CustomerID, c.CustomerName, c.Email
ORDER BY c.CustomerName;


## Question 8
**Retrieve ProductName, Price, StockQuantity, and CategoryName for all products. Order by CategoryName and then ProductName.**


In [ ]:
SELECT p.ProductName, p.Price, p.StockQuantity, c.CategoryName
FROM Products p
JOIN Categories c ON p.CategoryID = c.CategoryID
ORDER BY c.CategoryName, p.ProductName;


## Question 9
**Use a CTE and ROW_NUMBER() to show the top 2 most expensive products per category.**


In [ ]:
WITH RankedProducts AS (
  SELECT c.CategoryName, p.ProductName, p.Price,
    ROW_NUMBER() OVER (PARTITION BY c.CategoryName ORDER BY p.Price DESC) AS rn
  FROM Products p
  JOIN Categories c ON p.CategoryID = c.CategoryID
)
SELECT CategoryName, ProductName, Price FROM RankedProducts WHERE rn <= 2;


## Question 10 (Sakila Database)
SQL queries to answer the business questions using the Sakila sample database.


In [ ]:
-- 1) Top 5 customers by total amount spent
SELECT CONCAT(c.first_name, ' ', c.last_name) AS CustomerName, c.email, SUM(p.amount) AS TotalSpent
FROM payment p
JOIN customer c ON p.customer_id = c.customer_id
GROUP BY c.customer_id
ORDER BY TotalSpent DESC
LIMIT 5;


In [ ]:
-- 2) Top 3 movie categories by rental count
SELECT cat.name AS CategoryName, COUNT(*) AS RentalCount
FROM category cat
JOIN film_category fc USING (category_id)
JOIN inventory i USING (film_id)
JOIN rental r USING (inventory_id)
GROUP BY cat.category_id
ORDER BY RentalCount DESC
LIMIT 3;


In [ ]:
-- 3) Films available at each store and how many never rented
SELECT s.store_id,
  COUNT(i.inventory_id) AS TotalFilms,
  SUM(CASE WHEN r.rental_id IS NULL THEN 1 ELSE 0 END) AS NeverRented
FROM store s
JOIN inventory i ON s.store_id = i.store_id
LEFT JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY s.store_id;


In [ ]:
-- 4) Total revenue per month for the year 2023
SELECT DATE_FORMAT(payment_date, '%Y-%m') AS Month, SUM(amount) AS TotalRevenue
FROM payment
WHERE YEAR(payment_date) = 2023
GROUP BY Month
ORDER BY Month;


In [ ]:
-- 5) Customers who rented more than 10 times in the last 6 months
SELECT CONCAT(c.first_name, ' ', c.last_name) AS CustomerName, COUNT(r.rental_id) AS TotalRentals
FROM customer c
JOIN rental r ON c.customer_id = r.customer_id
WHERE r.rental_date >= DATE_SUB(CURDATE(), INTERVAL 6 MONTH)
GROUP BY c.customer_id
HAVING COUNT(r.rental_id) > 10;


---
### Notes
- The SQL code cells are intended to be executed in a SQL environment (MySQL / MariaDB / PostgreSQL with slight modifications).
- If you want, I can also provide a `.sql` file with all the queries or adapt this notebook to run SQL directly inside a Python kernel using an in-memory SQLite example.
